In [15]:
#Import packages

#Rdkit
from rdkit import Chem

#Scipy
from scipy import stats 
from scipy.stats import uniform,ranksums 

#Pandas
import pandas as pd
from pandas import DataFrame

#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib #jbolib模块
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\HIV")

In [3]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [4]:
Raw_data= pd.read_csv('./pubchem_targettaxid_12721_bioactivity.csv',index_col=0)
Raw_data.dropna(subset=["cid"], inplace=True)  
Raw_data['Activite']=np.where(Raw_data['activity']=='Active', 1, 0)
Raw_data.sort_values(by="Activite" , inplace=True, ascending=False) 
Raw_data.drop_duplicates(subset='cid', keep='first', inplace=True)
Raw_data

,acvalue,aid,sid,cid,geneid,pmid,aidtype,aidmdate,hasdrc,rnai,...,targetname,targeturl,ecs,repacxn,taxid,cellids,targettaxid,anatomyid,anatomy,Activite
baid,,,,,,,,,,,,,,,,,,,,,
101603994,1.600000e-07,399882,103262749,395575,NaN,14987069.0,Confirmatory,20220830,0,0,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,1
101772343,1.600000e-01,313900,103560697,15604702,NaN,18083561.0,Confirmatory,20220830,0,0,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,7578.0,12721,NaN,NaN,1
106035022,1.845000e-01,443684,103707224,16045493,NaN,19883100.0,Confirmatory,20220830,0,0,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,1
367492413,1.800000e-01,1662506,442062568,146894788,NaN,32331932.0,Confirmatory,20220830,0,0,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,1
334019337,1.800000e-01,722191,164136782,69480289,NaN,23245515.0,Confirmatory,20220830,0,0,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100881339,1.581250e+05,373107,103633261,44582935,NaN,18687505.0,Confirmatory,20180910,0,0,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,0
100881346,1.559550e+05,373107,103633775,44583099,NaN,18687505.0,Confirmatory,20180910,0,0,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,0
100881244,1.555970e+05,373107,103633653,44583063,NaN,18687505.0,Confirmatory,20180910,0,0,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,0


In [5]:
Raw_data.index=Raw_data['cid']
Raw_data=Raw_data.drop(['cid'], axis=1)
print_RxC(Raw_data)
Raw_data

1919 Moleculars × 26 Descriptors


,acvalue,aid,sid,geneid,pmid,aidtype,aidmdate,hasdrc,rnai,activity,...,targetname,targeturl,ecs,repacxn,taxid,cellids,targettaxid,anatomyid,anatomy,Activite
cid,,,,,,,,,,,,,,,,,,,,,
395575,1.600000e-07,399882,103262749,NaN,14987069.0,Confirmatory,20220830,0,0,Active,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,1
15604702,1.600000e-01,313900,103560697,NaN,18083561.0,Confirmatory,20220830,0,0,Active,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,7578.0,12721,NaN,NaN,1
16045493,1.845000e-01,443684,103707224,NaN,19883100.0,Confirmatory,20220830,0,0,Active,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,1
146894788,1.800000e-01,1662506,442062568,NaN,32331932.0,Confirmatory,20220830,0,0,Active,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,1
69480289,1.800000e-01,722191,164136782,NaN,23245515.0,Confirmatory,20220830,0,0,Active,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44582935,1.581250e+05,373107,103633261,NaN,18687505.0,Confirmatory,20180910,0,0,Unspecified,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,0
44583099,1.559550e+05,373107,103633775,NaN,18687505.0,Confirmatory,20180910,0,0,Unspecified,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,0
44583063,1.555970e+05,373107,103633653,NaN,18687505.0,Confirmatory,20180910,0,0,Unspecified,...,Human immunodeficiency virus,/taxonomy/12721,NaN,NaN,NaN,NaN,12721,NaN,NaN,0


In [6]:
Smiles_data= pd.read_csv('./data.csv',index_col=0)
Smiles_data['cid']=Smiles_data.index
Smiles_data.drop_duplicates(subset='cid', keep='first', inplace=True)

Smiles_data=Smiles_data.drop(['cid'], axis=1)
print_RxC(Smiles_data)
Smiles_data.head()

1919 Moleculars × 7 Descriptors


,IUPACName,IsomericSMILES,MolecularFormula,MolecularWeight,HBondDonorCount,HBondAcceptorCount,Synonym
CID,,,,,,,
395575,"[(9R,10R)-4,8,8-trimethyl-2-oxo-9-(4,7,7-trime...",CC1=CC(=O)OC2=C1C=CC3=C2[C@H]([C@H](C(O3)(C)C)...,C35H40O11,636.70,0,11,"['4-Methyl-DCK', 'NSC700281', '[(9R,10R)-4,8,8..."
135398508,"2-amino-9-[(1S,3R,4S)-4-hydroxy-3-(hydroxymeth...",C=C1[C@H](C[C@@H]([C@H]1CO)O)N2C=NC3=C2N=C(NC3...,C12H15N5O3,277.28,4,5,"['Entecavir', '142217-69-4', 'Baraclude', 'anh..."
44555481,"(5S)-5-butyl-9-[1-(4,6-dimethylpyrimidine-5-ca...",CCCC[C@H]1CN(C(=O)OC12CCN(CC2)C3(CCN(CC3)C(=O)...,C31H49N5O4,555.80,0,7,"['PharmaGSID_48521', 'CHEMBL1078231', 'DTXSID8..."
457928,"(1R,3aS,5aR,5bR,7aR,9S,11aR,11bR,13aR,13bR)-9-...",CC(=C)[C@@H]1CC[C@]2([C@H]1[C@H]3CC[C@@H]4[C@]...,C36H56O6,584.80,2,6,"['Bevirimat', '174022-42-5', 'PA-457', 'Beviri..."
380985,"[(9R,10R)-8,8-dimethyl-2-oxo-9-(4,7,7-trimethy...",CC1([C@@H]([C@@H](C2=C(O1)C=CC3=C2OC(=O)C=C3)O...,C34H38O11,622.70,0,11,"['NSC666863', 'Pyranocoumarin camphanoyl deriv..."


In [7]:
Original_data=pd.DataFrame(Raw_data['Activite'])
Original_data['IsomericSMILES']=Smiles_data['IsomericSMILES']

Chemdraw_smiles=Original_data['IsomericSMILES'].tolist()
print_RxC(Original_data)
Original_data.head()

1919 Moleculars × 2 Descriptors


,Activite,IsomericSMILES
cid,,
395575,1,CC1=CC(=O)OC2=C1C=CC3=C2[C@H]([C@H](C(O3)(C)C)...
15604702,1,CCCNS(=O)(=O)C1=C(NC2=C1C=C(C=C2)Br)C(=O)N
16045493,1,C1=CC(=NC2=NNC(=C21)COC3=CC(=C(C(=C3)F)Cl)OC4=...
146894788,1,CCC(=O)CCCCC[C@@H](C1=NC=C(N1)C2=CC3=CC=CC=C3N...
69480289,1,CN1C2=C(C(=C(C1=O)C(=O)NC3(CCCC3)CO)O)N=CC(=C2...


In [8]:
Smis=[]
for i in Chemdraw_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Original_data['Canonical_smiles']=Canonical_smis
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
print_RxC(Original_data)
Original_data

1919 Moleculars × 3 Descriptors


,Activite,IsomericSMILES,Canonical_smiles
cid,,,
395575,1,CC1=CC(=O)OC2=C1C=CC3=C2[C@H]([C@H](C(O3)(C)C)...,Cc1cc(=O)oc2c3c(ccc12)OC(C)(C)[C@H](OC(=O)C12C...
15604702,1,CCCNS(=O)(=O)C1=C(NC2=C1C=C(C=C2)Br)C(=O)N,CCCNS(=O)(=O)c1c(C(N)=O)[nH]c2ccc(Br)cc12
16045493,1,C1=CC(=NC2=NNC(=C21)COC3=CC(=C(C(=C3)F)Cl)OC4=...,N#Cc1cc(Cl)cc(Oc2cc(OCc3[nH]nc4nc(N)ccc34)cc(F...
146894788,1,CCC(=O)CCCCC[C@@H](C1=NC=C(N1)C2=CC3=CC=CC=C3N...,CCC(=O)CCCCC[C@H](NC(=O)[C@H]1CC12CCN(Cc1ccc(O...
69480289,1,CN1C2=C(C(=C(C1=O)C(=O)NC3(CCCC3)CO)O)N=CC(=C2...,Cn1c(=O)c(C(=O)NC2(CO)CCCC2)c(O)c2ncc(Cc3ccc(F...
...,...,...,...
44582935,0,CC1=C(C(=CC=C1)C)C2N(C(=O)CS2)C3=NC=CC(=C3)Br,Cc1cccc(C)c1C1SCC(=O)N1c1cc(Br)ccn1
44583099,0,CCC1CCC1N2C(SCC2=O)C3=C(C=CC=C3Cl)Cl,CCC1CCC1N1C(=O)CSC1c1c(Cl)cccc1Cl
44583063,0,CC1=CC(=C(N1)N2C(SCC2=S)C3=C(C=CC=C3F)F)C,Cc1cc(C)c(N2C(=S)CSC2c2c(F)cccc2F)[nH]1


In [9]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [11]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis)
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_gelation=aDesc.get_output()

In [12]:
X_original_data=pd.DataFrame(Data_gelation)
Desc_names_names=aDesc.get_output_descriptors()
X_original_data.columns=Desc_names_names
X_original_data.index=Original_data.index

X_original_data.head()

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
cid,,,,,,,,,,,,,,,,,,,,,
395575,636.75,7.404070,53.3988,87.2723,55.2275,96.6074,0.620916,1.014794,0.642180,1.123342,...,0.0,1.333333,0.0,5.0,39.333333,0.855072,9.75,0.211957,50.709583,12.622331
15604702,360.26,10.595882,22.5817,34.8944,23.9490,38.3759,0.664168,1.026306,0.704382,1.128703,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
16045493,444.27,10.577857,31.2089,43.7417,31.3978,47.2140,0.743069,1.041469,0.747567,1.124143,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
146894788,637.90,6.786170,57.0290,93.3738,60.8359,106.0483,0.606691,0.993338,0.647190,1.128173,...,0.0,0.000000,11.0,11.0,41.000000,0.872340,8.50,0.180851,39.957758,9.293476
69480289,425.50,7.736364,35.1015,55.8469,36.1480,62.2394,0.638209,1.015398,0.657236,1.131625,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [13]:
#Remove descriptors with excessive NA values
Thresh =int(0.8*len(X_original_data))
X_NAomit_data=X_original_data.dropna(
    #thresh=Thresh, 
    axis=1) 
X_NAomit_data.head()
X_NAomit_data.to_csv("./X_NAomit_data.csv",sep=',',header=1,index=1)

In [16]:
#Ranksum test
Ranksums_data=X_NAomit_data
Ranksums_data["Activity"]=Original_data['Activite']
Notgelable=Ranksums_data.loc[Ranksums_data["Activity"] == 0]
Gelable=Ranksums_data.loc[Ranksums_data["Activity"] == 1]
Ranksums_result=[]
for i in list(Ranksums_data.columns):
 NG = Notgelable[i]
 G = Gelable[i]
 Ranksums_caculate=ranksums(NG, G, alternative='two-sided')
 Ranksums_result.append(Ranksums_caculate)
Ranksums_results = pd.DataFrame(Ranksums_result, columns=['Statistics','Pvalues'])
Ranksums_results.index=Ranksums_data.columns#
Ranksums_results = Ranksums_results.drop('Activity',axis = 0)
Ranksums_results.sort_values(by="Pvalues" , inplace=True, ascending=True)
Ranksums_results

,Statistics,Pvalues
IC2,-17.283965,6.212693e-67
IC3,-17.000314,8.168187e-65
IDDE,-16.961287,1.588263e-64
F05[N-O],-15.653476,3.145596e-55
IC4,-15.600479,7.224973e-55
...,...,...
B09[B-X],0.000000,1.000000e+00
B09[B-Si],0.000000,1.000000e+00
B09[B-B],0.000000,1.000000e+00
B09[I-B],0.000000,1.000000e+00


In [17]:
#Ranksum test <0.05
Ranksums_marked = Ranksums_results[Ranksums_results['Pvalues'] <0.05]
print(Ranksums_marked.shape)
Ranksums_marked.head()

(1590, 2)


,Statistics,Pvalues
IC2,-17.283965,6.212693e-67
IC3,-17.000314,8.168187e-65
IDDE,-16.961287,1.588263e-64
F05[N-O],-15.653476,3.145596e-55
IC4,-15.600479,7.224973e-55


In [18]:
X_ranksums_data=X_NAomit_data[Ranksums_marked.index]
X_ranksums_data.head()

,IC2,IC3,IDDE,F05[N-O],IC4,Vindex,Xindex,SHED_AL,IC5,SM6_B(v),...,GATS3i,MATS7i,SpMAD_EA(bo),CATS2D_02_AP,nS,GATS8v,s2_phRelSize_2,C-033,Eta_epsi_4,s2_pathLength
cid,,,,,,,,,,,,,,,,,,,,,
395575,4.370521,4.975172,5.306171,0.0,5.021684,0.159076,0.239194,10.723942,5.091451,9.646311,...,1.289058,0.037151,1.842058,0.0,0.0,1.092054,0.285859,0.0,0.585461,3.333333
15604702,4.572469,4.712319,4.121928,3.0,4.712319,0.367510,0.535952,5.125172,4.712319,9.050349,...,1.043726,-0.070692,2.076232,1.0,1.0,1.313248,0.000000,0.0,0.581746,0.000000
16045493,4.781568,5.297079,4.640224,2.0,5.297079,0.174688,0.277373,11.757815,5.297079,9.383009,...,1.055406,-0.024600,2.047847,1.0,0.0,0.893486,0.000000,0.0,0.610673,0.000000
146894788,5.128025,5.807092,5.214163,3.0,5.934752,0.117592,0.189401,17.209792,5.934752,9.629762,...,1.155428,-0.020229,1.829185,0.0,0.0,0.961788,0.055556,0.0,0.516970,4.000000
69480289,4.792914,5.113089,4.438067,2.0,5.113089,0.188201,0.294630,10.665381,5.113089,9.233977,...,1.156548,0.008156,1.869996,0.0,0.0,1.150586,0.000000,0.0,0.565387,0.000000


In [19]:
#Scale data
Scaler = preprocessing.MinMaxScaler() #StandardScaler
Transformer =Scaler.fit(X_ranksums_data)
X_scaled_data=Transformer.transform(X_ranksums_data)
X_scaled_data =pd.DataFrame(X_scaled_data)
X_scaled_data.columns=X_ranksums_data.columns
X_scaled_data.index=Original_data.index
X_scaled_data.to_csv("./X_scaled_data.csv",sep=',',header=1,index=1)
joblib.dump(Transformer, './Scaler_transformer.pkl')
X_scaled_data.head()

,IC2,IC3,IDDE,F05[N-O],IC4,Vindex,Xindex,SHED_AL,IC5,SM6_B(v),...,GATS3i,MATS7i,SpMAD_EA(bo),CATS2D_02_AP,nS,GATS8v,s2_phRelSize_2,C-033,Eta_epsi_4,s2_pathLength
cid,,,,,,,,,,,,,,,,,,,,,
395575,0.632912,0.647209,0.524016,0.000000,0.574759,0.001753,0.002566,0.587693,0.535935,0.488822,...,0.664773,0.386765,0.622626,0.000000,0.00,0.453823,0.285859,0.0,0.397836,0.416667
15604702,0.687508,0.583118,0.321200,0.049180,0.508924,0.004050,0.005750,0.280869,0.462597,0.352709,...,0.510544,0.309712,0.858462,0.111111,0.25,0.545744,0.000000,0.0,0.387818,0.000000
16045493,0.744037,0.725700,0.409965,0.032787,0.633365,0.001925,0.002976,0.644351,0.575711,0.428686,...,0.517887,0.342645,0.829876,0.111111,0.00,0.371304,0.000000,0.0,0.465825,0.000000
146894788,0.837700,0.850057,0.508259,0.049180,0.769066,0.001296,0.002032,0.943130,0.699059,0.485042,...,0.580766,0.345767,0.609660,0.000000,0.00,0.399689,0.055556,0.0,0.213139,0.500000
69480289,0.747104,0.680838,0.375343,0.032787,0.594211,0.002074,0.003161,0.584483,0.540120,0.394648,...,0.581470,0.366048,0.650762,0.000000,0.00,0.478147,0.000000,0.0,0.343705,0.000000


In [20]:
#Remove high correlation descriptors
X_cor_data=trimm_correlated(X_scaled_data,0.80)
X_cor_data.index=Original_data.index
X_cor_data.to_csv("./ML_data.csv",sep=',',header=1,index=1)
print_RxC(X_cor_data)
X_cor_data.head()

1919 Moleculars × 277 Descriptors


,IC2,F05[N-O],Vindex,SHED_AL,SM6_B(v),CATS2D_05_AA,F08[C-N],F03[N-O],P_VSA_LogP_4,B10[C-N],...,B08[N-S],CATS2D_01_DA,B05[O-S],GATS5m,GATS5e,nCconj,MATS7i,CATS2D_02_AP,GATS8v,C-033
cid,,,,,,,,,,,,,,,,,,,,,
395575,0.632912,0.000000,0.001753,0.587693,0.488822,0.173913,0.000000,0.00000,0.085719,0.0,...,0.0,0.00,0.0,0.481970,0.666269,0.230769,0.386765,0.000000,0.453823,0.0
15604702,0.687508,0.049180,0.004050,0.280869,0.352709,0.043478,0.005208,0.02439,0.039319,0.0,...,0.0,0.00,0.0,0.357018,0.601275,0.076923,0.309712,0.111111,0.545744,0.0
16045493,0.744037,0.032787,0.001925,0.644351,0.428686,0.043478,0.036458,0.02439,0.026535,1.0,...,0.0,0.25,0.0,0.429851,0.431305,0.000000,0.342645,0.111111,0.371304,0.0
146894788,0.837700,0.049180,0.001296,0.943130,0.485042,0.065217,0.083333,0.00000,0.057152,1.0,...,0.0,0.00,0.0,0.479589,0.481747,0.000000,0.345767,0.000000,0.399689,0.0
69480289,0.747104,0.032787,0.002074,0.584483,0.394648,0.065217,0.020833,0.04878,0.069420,1.0,...,0.0,0.00,0.0,0.517585,0.536060,0.307692,0.366048,0.000000,0.478147,0.0
